<a href="https://colab.research.google.com/github/davidelgas/DataSciencePortfolio/blob/main/OCCP/OCCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare Enviornment

Organization < Property < Location < Cluster < Station < UserID

A cluster is a grouping of chargers/stations. This for convenience/load balancing

Each circuit can have multiple clusters.

Each cluster has its own breaker


In [1]:
# Access to Google Drive
# This seems to propagate credentials better from its own cell

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pandas pyxlsb
import pandas as pd

import numpy as np

import os
import logging
import psycopg2

!pip install SQLAlchemy psycopg2-binary
import seaborn as sns
import matplotlib.pyplot as p

import json

import statsmodels.api as sm
from statsmodels.formula.api import ols

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import matplotlib.pyplot as plt

from datetime import timedelta
import holidays

!pip install statsmodels
import statsmodels.api as sm


## Ingest data

In [3]:
# Load Log files and concatinate

def load_file(file_path):
    """Load a CSV file into a Pandas DataFrame."""
    try:
        df = pd.read_csv(file_path)
        print(f"Loaded {file_path}: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on failure

def save_to_csv(df, output_path):
    """Save DataFrame to a CSV file."""
    try:
        df.to_csv(output_path, index=False)
        print(f"Saved concatenated file to {output_path}")
    except Exception as e:
        print(f"Error saving file {output_path}: {e}")

# Main Function: Concatenate Files
def concatenate_files(file_paths, output_file):
    """
    Load multiple files, concatenate them, and save as a single file.
    """
    combined_df = pd.DataFrame()

    for file_path in file_paths:
        print(f"Loading file: {file_path}")
        df = load_file(file_path)

        if not df.empty:
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        else:
            print(f"Skipping empty file: {file_path}")

    print(f"Combined DataFrame shape: {combined_df.shape}")
    save_to_csv(combined_df, output_file)

# File Paths and Configuration
file_paths = [
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/sept_100_sample.csv',
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/oct_100_sample.csv',
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/nov_100_sample.csv'
]

output_file = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/combined_data.csv'

# Run Concatenation
concatenate_files(file_paths, output_file)

print("All files concatenated and saved successfully.")


Loading file: /content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/sept_100_sample.csv
Loaded /content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/sept_100_sample.csv: (425918, 4)
Loading file: /content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/oct_100_sample.csv
Loaded /content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/oct_100_sample.csv: (451574, 4)
Loading file: /content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/nov_100_sample.csv
Loaded /content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/nov_100_sample.csv: (444630, 4)
Combined DataFrame shape: (1322122, 4)
Saved concatenated file to /content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/combined_data.csv
All files concatenated and saved successfully.


In [4]:
# Load Properties table

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    """Load credentials and set as environment variables."""
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Export PostgreSQL table to CSV
def export_table_to_csv(connection_params, table_name, output_csv_path):
    """Export table data from PostgreSQL to CSV."""
    try:
        # Connect to the database
        connection = psycopg2.connect(**connection_params)
        cursor = connection.cursor()

        # Query to fetch all rows
        query = f"SELECT * FROM {table_name};"
        cursor.execute(query)

        # Fetch rows and column headers
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]

        # Create DataFrame
        df = pd.DataFrame(rows, columns=column_names)
        logging.info(f"Loaded table '{table_name}' with shape: {df.shape}")

        # Save raw table data to CSV
        df.to_csv(output_csv_path, index=False)
        logging.info(f"Data exported to {output_csv_path} successfully.")

    except Exception as e:
        logging.error(f"Error processing table '{table_name}': {str(e)}")

    finally:
        if 'connection' in locals() and connection:
            cursor.close()
            connection.close()
            logging.info("Database connection closed.")

# Configuration
tables = ["properties"]
output_csv_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/properties.csv'

# Process tables
for table in tables:
    export_table_to_csv(connection_params, table, output_csv_path)


## Clean data

In [ ]:
# Expand the JSON data in the logs
import pandas as pd
import json
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_csv(file_path):
    """
    Load a CSV file into a DataFrame.
    """
    try:
        df = pd.read_csv(file_path)
        logging.info(f"Loaded file: {file_path}, Shape: {df.shape}")
        return df
    except Exception as e:
        logging.error(f"Error loading file {file_path}: {e}")
        return pd.DataFrame()

def flatten_row(row, json_column):
    """
    Helper function to flatten a single row with JSON data.
    """
    try:
        if pd.isna(row[json_column]):
            return []

        # Extract base data
        base_data = row.drop(json_column).to_dict()

        # Parse the JSON column
        json_data = json.loads(row[json_column])

        flattened_rows = []
        for meter_value in json_data.get("meterValue", []):
            # Extract timestamp
            timestamp = meter_value.get("timestamp", None)

            for i, sampled_value in enumerate(meter_value.get("sampledValue", [])):
                # Extract only the required fields
                flat_row = {
                    **base_data,
                    "timestamp": timestamp,
                    f"value_{i}": sampled_value.get("value", None),
                    f"unit_{i}": sampled_value.get("unit", None)
                }
                flattened_rows.append(flat_row)
        return flattened_rows
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        logging.warning(f"Error processing row: {e}")
        return []

def flatten_column(df, json_column):
    """
    Flatten a JSON column in a DataFrame.
    """
    flattened_rows = []
    for index, row in df.iterrows():
        flattened_rows.extend(flatten_row(row, json_column))
    return pd.DataFrame(flattened_rows)

def save_to_csv(df, output_file):
    """
    Save a DataFrame to a CSV file.
    """
    try:
        os.makedirs(os.path.dirname(output_file), exist_ok=True)  # Ensure the output directory exists
        df.to_csv(output_file, index=False)
        logging.info(f"Saved file to: {output_file}")
    except Exception as e:
        logging.error(f"Error saving file {output_file}: {e}")

def process_and_flatten(file_path, json_column, output_dir):
    """
    Load a file, flatten its JSON column, and save the output.
    """
    df = load_csv(file_path)
    if df.empty:
        logging.warning(f"File {file_path} is empty. Skipping.")
        return

    if json_column not in df.columns:
        logging.error(f"Column {json_column} does not exist in the DataFrame.")
        return

    flattened_df = flatten_column(df, json_column)
    if not flattened_df.empty:
        # Construct the output file path
        output_file = os.path.join(output_dir, f"{os.path.basename(file_path).split('.')[0]}_expanded.csv")
        save_to_csv(flattened_df, output_file)
    else:
        logging.warning(f"No data left after flattening {json_column}. Skipping save.")

# Example Usage
if __name__ == "__main__":
    file_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/combined_data.csv'
    json_column = 'cleaned_message'
    output_dir = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP'

    process_and_flatten(file_path, json_column, output_dir)


In [ ]:
# Join the two

# File paths
combined_data_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/combined_data_flattened.csv'
properties_data_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/properties.csv'
output_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/combined_with_properties.csv'

def left_join_data(combined_path, properties_path, output_path):
    """
    Perform a left join between combined_data.csv and properties.csv.
    - combined_data.property_id joins properties.id.
    """
    try:
        # Load datasets
        combined_df = pd.read_csv(combined_path)
        properties_df = pd.read_csv(properties_path)
        print(f"Loaded combined_data: {combined_df.shape}")
        print(f"Loaded properties: {properties_df.shape}")

        # Perform left join
        result_df = pd.merge(
            combined_df,
            properties_df,
            left_on='property_id',
            right_on='id',
            how='left'
        )

        # Save the result
        result_df.to_csv(output_path, index=False)
        print(f"Saved joined data to {output_path}")
        print(f"Final DataFrame shape: {result_df.shape}")

    except Exception as e:
        print(f"Error during join: {e}")

# Run the join operation
left_join_data(combined_data_path, properties_data_path, output_path)


In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/OCCP/combined_with_properties.csv')

fields_to_keep = [
'timestamp',
'property_id',
'property_type',
'user_id',
'value_0',
'unit_0',
'unit_1',
'value_1'
]

df = df_1[fields_to_keep]





In [ ]:
df.info()

In [ ]:
# Clean data

# UDF for cleaning the 'timestamp' column
def clean_timestamp(df, column_name="timestamp"):
    """
    Cleans the timestamp column:
    - Converts to datetime.
    - Drops rows with invalid or null timestamps.
    """
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    df_cleaned = df.dropna(subset=[column_name])
    return df_cleaned

# UDF for cleaning the 'property_id' column
def clean_property_id(df, column_name="property_id"):
    """
    Cleans the property_id column:
    - Drops rows where 'property_id' is null.
    """
    df_cleaned = df.dropna(subset=[column_name])
    return df_cleaned

# UDF for cleaning 'value_0' column
def clean_value_0(df, column_name="value_0"):
    """
    Cleans the value_0 column:
    - Replaces NaN/Null values with 0.
    - Ensures the column data type is int.
    """
    df[column_name] = df[column_name].fillna(0).astype(int)
    return df

# UDF for cleaning 'value_1' column
def clean_value_1(df, column_name="value_1"):
    """
    Cleans the value_1 column:
    - Replaces NaN/Null values with 0.
    - Ensures the column data type is int.
    """
    df[column_name] = df[column_name].fillna(0).astype(int)
    return df

# Generalized function to clean a DataFrame using a set of UDFs
def clean_data_with_udfs(df, cleaning_functions):
    """
    Applies a sequence of cleaning UDFs to the DataFrame.
    :param df: Input DataFrame.
    :param cleaning_functions: List of (function, column_name) tuples.
    """
    for func, col in cleaning_functions:
        print(f"Applying cleaning rule: {func.__name__} on column: {col}")
        df = func(df, column_name=col)
    return df

# File paths
file_paths = [
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/sept_100_sample_flat.csv',
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/oct_100_sample_flat.csv',
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/nov_100_sample_flat.csv'
]

# Define cleaning rules as a list of (UDF, column_name) pairs
cleaning_rules = [
    (clean_timestamp, "timestamp"),
    (clean_property_id, "property_id"),  # Updated to property_id
    (clean_value_0, "value_0"),
    (clean_value_1, "value_1")
]

# Process each file
for file_path in file_paths:
    print(f"Processing file: {file_path}")
    df = pd.read_csv(file_path)  # Load the data

    # Apply cleaning rules
    df_cleaned = clean_data_with_udfs(df, cleaning_rules)

    # Save cleaned data
    output_path = file_path.replace(".csv", "_cleaned.csv")
    df_cleaned.to_csv(output_path, index=False)

    print(f"Cleaned file saved to: {output_path}")
    print(f"Rows before cleaning: {len(df)}, Rows after cleaning: {len(df_cleaned)}\n")


## Engineer Features

In [ ]:
import pandas as pd
import os

# Utility Functions
def add_day_info(df, timestamp_col='timestamp'):
    """
    Add day information including:
    - Day of the week as integers (Sunday=1, Monday=2, ..., Saturday=7)
    - Weekend indicator (1 if weekend, 0 otherwise)
    """
    # Extract day of the week (Sunday=1, Monday=2, ..., Saturday=7)
    df['day_of_week'] = df[timestamp_col].dt.dayofweek + 1  # Convert 0-6 (Monday-Sunday) to 1-7 (Sunday-Saturday)

    # Add weekend indicator
    df['day_weekend'] = (df['day_of_week'] >= 6).astype(int)  # Weekend: Saturday (6) and Sunday (7)

    return df

def calculate_days_to_nearest_holiday(df, date_col, holiday_dates):
    """
    Calculate the minimum number of days to the nearest holiday for each date.

    Parameters:
    - df: DataFrame containing the date column
    - date_col: Name of the column with datetime values
    - holiday_dates: List of holiday dates (as datetime objects)

    Returns:
    - DataFrame with a new column 'days_to_nearest_holiday'
    """
    # Ensure date column is in datetime format
    df[date_col] = pd.to_datetime(df[date_col])

    # Normalize holidays to match the timezone-awareness of the column
    if df[date_col].dt.tz is not None:
        # Make holidays timezone-aware if timestamps are timezone-aware
        holiday_dates = [pd.Timestamp(holiday).tz_localize(df[date_col].dt.tz) for holiday in holiday_dates]
    else:
        # Ensure holidays are timezone-naive if timestamps are timezone-naive
        holiday_dates = [pd.Timestamp(holiday).tz_localize(None) for holiday in holiday_dates]

    # Calculate days to each holiday and find the minimum
    df['days_to_nearest_holiday'] = df[date_col].apply(
        lambda x: min(abs((x - holiday).days) for holiday in holiday_dates)
    )

    return df

def add_datetime_components(df, timestamp_col='timestamp'):
    """Add year, month, day, and hour components as separate columns."""
    df['year'] = df[timestamp_col].dt.year
    df['month'] = df[timestamp_col].dt.month
    df['day'] = df[timestamp_col].dt.day
    df['hour'] = df[timestamp_col].dt.hour
    return df


def encode_month_column(df, month_col='month'):
    """
    Encode the month column as integers (September=9, October=10, November=11).
    """
    # Map month names (if needed) to integers
    month_mapping = {'September': 9, 'October': 10, 'November': 11}
    df['month_encoded'] = df[month_col].map(month_mapping).fillna(df[month_col]).astype(int)

    return df


def add_unique_user_counts(df, group_cols, user_col='user_id'):
    """Add a column with the count of unique users per group."""
    unique_user_counts = (
        df.groupby(group_cols)[user_col]
        .nunique()
        .reset_index()
        .rename(columns={user_col: 'unique_user_count'})
    )
    df = df.merge(unique_user_counts, on=group_cols, how='left')
    return df


def add_usage_sums(df, group_cols, value_cols):
    """Add sum of usage per group."""
    sums = df.groupby(group_cols)[value_cols].sum().reset_index()
    sums.rename(
        columns={
            value_cols[0]: 'hour_sum_value_Wh',
            value_cols[1]: 'hour_sum_value_A'
        },
        inplace=True
    )
    df = df.merge(sums, on=group_cols, how='left')
    return df


def engineer_data(df, timestamp_col, user_col, group_cols, value_cols, holiday_dates=None):
    """
    Engineer new features for the dataset:
    - Add day information
    - Add holiday proximity
    - Extract datetime components
    - Add unique user counts
    - Add usage sums
    """
    # Ensure timestamp is in datetime format
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce', utc=True)

    # Add day information
    df = add_day_info(df, timestamp_col)

    # Add holiday proximity if holiday dates are provided
    if holiday_dates:
        df = calculate_days_to_nearest_holiday(df, timestamp_col, holiday_dates)

    # Extract datetime components
    df = add_datetime_components(df, timestamp_col)

    # Flatten Month
    df = encode_month_column(df)

    # Add unique user counts
    df = add_unique_user_counts(df, group_cols, user_col)

    # Add usage sums
    df = add_usage_sums(df, group_cols, value_cols)

    return df

def process_files(file_paths, output_dir, group_cols, value_cols, timestamp_col='timestamp', user_col='user_id', holiday_dates=None, properties_file=None):
    """
    Process a list of files:
    - Load CSV files
    - Engineer features
    - Merge metadata from properties CSV
    - Save processed files to an output directory
    """
    processed_dfs = []

    # Load properties metadata if provided
    if properties_file:
        properties_df = pd.read_csv(properties_file)
        print(f"Loaded properties metadata from {properties_file}")

    for file_path in file_paths:
        # Extract file name without extension
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = os.path.join(output_dir, f"{file_name}_processed.csv")

        print(f"Processing file: {file_path}")

        # Load the CSV file
        df = pd.read_csv(file_path)

        # Apply feature engineering
        processed_df = engineer_data(df, timestamp_col, user_col, group_cols, value_cols, holiday_dates)

        # Merge with properties metadata if available
        if properties_file:
            processed_df = pd.merge(processed_df, properties_df, left_on='property_id', right_on='id', how='left')
            print(f"Metadata merged for {file_name}")

        # Save the processed DataFrame
        processed_df.to_csv(output_file, index=False)
        print(f"Processed file saved to: {output_file}")

        # Append to list of processed DataFrames
        processed_dfs.append(processed_df)

    return processed_dfs


# Example Usage
file_paths = [
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/sept_100_sample_flat.csv',
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/oct_100_sample_flat.csv',
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/nov_100_sample_flat.csv'
]

properties_file = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/properties.csv'  # Path to properties CSV
output_dir = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/'

group_cols = ['property_id', 'year', 'month', 'day', 'hour']
value_cols = ['value_0', 'value_1']
holiday_dates = [
    pd.Timestamp("2024-09-04"),
    pd.Timestamp("2024-10-09"),
    pd.Timestamp("2024-11-23")
]

processed_dfs = process_files(
    file_paths=file_paths,
    output_dir=output_dir,
    group_cols=group_cols,
    value_cols=value_cols,
    timestamp_col='timestamp',
    user_col='user_id',
    holiday_dates=holiday_dates,
    properties_file=properties_file  # Pass properties file here
)

# Combine all processed DataFrames into one
final_combined_df = pd.concat(processed_dfs, axis=0).reset_index(drop=True)

# Save the combined DataFrame
final_combined_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/final_combined.csv', index=False)

print("Processing complete. Combined DataFrame saved.")


In [ ]:
df.info()

In [ ]:
#Ingest log tables that were exported from Splunk


# Utility Functions
def load_file(file_path):
    """Load a CSV file into a Pandas DataFrame."""
    try:
        df = pd.read_csv(file_path)
        print(f"Loaded {file_path}: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on failure


def flatten_column(df, json_column):
    """Flatten a JSON column in a DataFrame."""
    flattened_rows = []
    for index, row in df.iterrows():
        try:
            if pd.isna(row[json_column]):
                continue  # Skip rows where the JSON column is missing or NaN

            base_data = row.drop(json_column).to_dict()  # Base columns (_time, property_id, user_id)
            json_data = json.loads(row[json_column])  # Parse JSON

            for meter_value in json_data.get("meterValue", []):
                timestamp = meter_value.get("timestamp", None)
                for i, sampled_value in enumerate(meter_value.get("sampledValue", [])):
                    flat_row = {
                        **base_data,
                        "timestamp": timestamp,
                        f"value_{i}": sampled_value.get("value", None),
                        f"context_{i}": sampled_value.get("context", None),
                        f"format_{i}": sampled_value.get("format", None),
                        f"measurand_{i}": sampled_value.get("measurand", None),
                        f"phase_{i}": sampled_value.get("phase", None),
                        f"location_{i}": sampled_value.get("location", None),
                        f"unit_{i}": sampled_value.get("unit", None),
                    }
                    flattened_rows.append(flat_row)
        except (json.JSONDecodeError, KeyError, TypeError) as e:
            print(f"Error processing row {index}: {e}")
    flattened_df = pd.DataFrame(flattened_rows)
    print(f"Flattened DataFrame shape: {flattened_df.shape}")
    return flattened_df


def drop_missing_rows(df, required_columns):
    """Drop rows with missing values (NaN, Null, or inf) in the specified columns."""
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"Warning: Missing required columns {missing_columns}. Skipping drop.")
        return df

    print(f"Checking for missing values in columns: {required_columns}")
    initial_row_count = len(df)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.dropna(subset=required_columns).copy()
    final_row_count = len(df)
    print(f"Dropped {initial_row_count - final_row_count} rows with missing values.")
    return df


def clean_data(df):
    """Clean all data by handling NaN and inf globally."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    return df


def clean_parsing_errors(df, unit_0_column, unit_1_column):
    """Drop rows where unit_0 is not 'Wh' and unit_1 is not 'A'."""
    if unit_0_column not in df.columns or unit_1_column not in df.columns:
        print(f"Skipping parsing errors: Missing columns {unit_0_column} or {unit_1_column}.")
        return df

    conditions = (
        ((df[unit_0_column] == 'Wh') | (pd.isna(df[unit_0_column]))) &
        ((df[unit_1_column] == 'A') | (pd.isna(df[unit_1_column])))
    )
    initial_row_count = len(df)
    cleaned_df = df[conditions].copy()
    print(f"Cleaned parsing errors: {initial_row_count - len(cleaned_df)} rows dropped.")
    return cleaned_df


def filter_columns(df, fields_to_keep):
    """Keep only specified columns in the DataFrame."""
    missing_columns = [col for col in fields_to_keep if col not in df.columns]
    if missing_columns:
        print(f"Warning: Missing columns {missing_columns}. Skipping those.")
    return df[[col for col in fields_to_keep if col in df.columns]].copy()


def convert_to_datetime(df, columns):
    """Convert specified columns to datetime."""
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)
    return df


def convert_to_int(df, columns):
    """Convert specified columns to integers, replacing NaN with 0 and inf with large values."""
    for col in columns:
        if col in df.columns:
            # Replace NaN or inf/-inf with 0 or a large number as appropriate
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, coerce errors to NaN
            df[col] = df[col].replace([np.inf, -np.inf], 0)  # Replace infinities with 0 or another value
            df[col] = df[col].fillna(0)  # Fill NaN with 0
            df[col] = df[col].astype(int)  # Convert to int after replacing NaN and inf
    return df


def save_to_csv(df, output_path):
    """Save the DataFrame to a CSV file."""
    df.to_csv(output_path, index=False)


# Main Processing Function
def process_files(file_paths, json_column, fields_to_keep, output_dir):
    """
    Process a list of files:
    - Load
    - Flatten JSON
    - Drop rows with missing values
    - Clean parsing errors
    - Filter out columns
    - Convert datatypes
    - Save to CSV
    """
    for file_path in file_paths:
        print(f"Processing file: {file_path}")
        df = load_file(file_path)

        if df.empty:
            print(f"File {file_path} is empty. Skipping.")
            continue

        df = flatten_column(df, json_column)
        df = clean_data(df)
        df = drop_missing_rows(df, ['timestamp', 'user_id'])
        df = clean_parsing_errors(df, 'unit_0', 'unit_1')
        df = filter_columns(df, fields_to_keep)
        df = convert_to_datetime(df, ['timestamp', '_time'])

        if not df.empty:
            output_path = os.path.join(output_dir, f"{os.path.basename(file_path).split('.')[0]}_flat.csv")
            save_to_csv(df, output_path)
            print(f"Processed file saved to: {output_path}")
        else:
            print(f"No data left after processing {file_path}. Skipping.")


# File Paths and Configuration
file_paths = [
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/sept_100_sample.csv',
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/oct_100_sample.csv',
    '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/nov_100_sample.csv'
]

output_dir = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie'

# Trimming data here due to file sizes
fields_to_keep = ['timestamp', 'user_id', 'property_id', 'value_0', 'value_1', 'unit_0', 'unit_1']

# Process All Files
process_files(file_paths, 'cleaned_message', fields_to_keep, output_dir)

print("All files processed successfully.")


In [ ]:
df_son.head(5)

In [ ]:
# Drop columns I dont need

columns_to_drop = ['_time', 'property_id','user_id','timestamp','unit_0','value_0','unit_1','value_1','day_weekend','year','month','hour','hour_sum_value_A']

# Drop the columns from the DataFrame

df_son = final_combined_df.drop(columns=columns_to_drop)

# Convert object type columns with string 'True'/'False' to boolean
df_son[df_son.select_dtypes('object').columns] = df_son.select_dtypes('object').applymap(lambda x: x == 'True')

# Verify the changes
print(df_son.dtypes)  # To check if the columns are now boolean

# Save the combined DataFrame

df_son.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/df_son.csv', index=False)

## Check for colinearity

In [ ]:
# VIF
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


# Convert boolean columns to integers for VIF calculation
X = X.astype(int)

# Add a constant column to the DataFrame for the intercept
X_with_const = add_constant(X)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i + 1) for i in range(len(X.columns))]

# Display VIF
print(vif_data)


# Prep df for regression/ANOVA

In [ ]:
# Define predictors (X) and dependent variable (y)
X = df_son.drop(columns=['hour_sum_value_Wh'])  # All columns except the dependent variable
y = df_son['hour_sum_value_Wh']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Check types of X and y
print(X.dtypes)
print(y.dtypes)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression model
print(model.summary())


In [ ]:
import numpy as np
# Fit a regression model
X = sm.add_constant(df_sampled['unique_user_count'])  # Add intercept
model = sm.OLS(df_sampled['hour_sum_value_A'], X).fit()

# Predict values for regression line
predictions = model.predict(X)

# Plot scatter with regression line
plt.figure(figsize=(8, 6))
plt.scatter(df_sampled['unique_user_count'], df_sampled['hour_sum_value_A'], alpha=0.6, label='Data Points')
plt.plot(df_sampled['unique_user_count'], predictions, color='red', label='Regression Line')
plt.title('Regression Plot: Unique User Count vs Hour Sum Value 0')
plt.xlabel('Unique User Count')
plt.ylabel('Hour hour_sum_value_A')
plt.legend()
plt.show()


# Ensure property_id is treated as a categorical variable
df_sampled['property_id'] = df_sampled['property_id'].astype('category')

# Prepare the formula for ANOVA
independent_vars = ['unique_user_count', 'property_id'] + [col for col in df_sampled.columns if col.startswith('day_')]
formula = 'hour_sum_value_A ~ ' + ' + '.join(independent_vars)

# Fit the model
model = ols(formula, data=df_sampled).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model, typ=2)

# Display the ANOVA results
print(anova_results)


In [ ]:
# Calculate the IQR for the column with potential outliers
Q1 = df_sampled['hour_sum_value_A'].quantile(0.25)
Q3 = df_sampled['hour_sum_value_A'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
df_filtered = df_sampled[(df_sampled['hour_sum_value_A'] >= lower_bound) & (df_sampled['hour_sum_value_A'] <= upper_bound)]


In [ ]:
## Create a property lookup

import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Function to fetch non-transposed data
def fetch_non_transposed_data(cursor, table):
    query = f"SELECT * FROM {table};"
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(rows, columns=column_names)
    return df

# List of tables to process
tables = [
    "location",
    "properties"
]

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Fetch non-transposed data
        df_non_transposed = fetch_non_transposed_data(cursor, table)

        # Write the DataFrame to CSV with new naming convention
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/{table}_table_extract.csv'
        df_non_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")



In [ ]:
import numpy as np
# Fit a regression model
X = sm.add_constant(df_sampled['unique_user_count'])  # Add intercept
model = sm.OLS(df_sampled['hour_sum_value_A'], X).fit()

# Predict values for regression line
predictions = model.predict(X)

# Plot scatter with regression line
plt.figure(figsize=(8, 6))
plt.scatter(df_filtered['unique_user_count'], df_filtered['hour_sum_value_A'], alpha=0.6, label='Data Points')
plt.plot(df_filtered['unique_user_count'], predictions, color='red', label='Regression Line')
plt.title('Regression Plot: Unique User Count vs Hour Sum Value 0')
plt.xlabel('Unique User Count')
plt.ylabel('Hour hour_sum_value_A')
plt.legend()
plt.show()


# Ensure property_id is treated as a categorical variable
df_filtered['property_id'] = df_filtered['property_id'].astype('category')

# Prepare the formula for ANOVA
independent_vars = ['unique_user_count', 'property_id'] + [col for col in df_filtered.columns if col.startswith('day_')]
formula = 'hour_sum_value_A ~ ' + ' + '.join(independent_vars)

# Fit the model
model = ols(formula, data=df_filtered).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model, typ=2)

# Display the ANOVA results
print(anova_results)

## Using an extract from Eng

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/2024-08-01.csv'

df_Aug_1_24 = pd.read_csv(file_path)

In [ ]:
df_Aug_1_24.info()

Need confirmation that it is watts and not watt hours.

In [ ]:
# Adding or modifying the header
df_Aug_1_24.columns = ['qrcode','connector', 'serial_num', 'org_id', 'property_id', 'station_id', 'transaction_id', 'metered_type', 'timestamp', 'metered_value']
print(df_Aug_1_24)

## Start Here

In [ ]:
connector_id = message_data['connectorId']
transaction_id = message_data['transactionId']
meter_values = message_data['meterValue']

print(f"Connector ID: {connector_id}")
print(f"Transaction ID: {transaction_id}")

# Loop through meter values
for mv in meter_values:
    timestamp = mv['timestamp']
    print(f"Timestamp: {timestamp}")
    for sv in mv['sampledValue']:
        print(f"  Measurand: {sv['measurand']}, Value: {sv['value']}, Unit: {sv['unit']}")


In [ ]:
# Aggregate all three months of data

file_path_a = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/e95523e6-3470-4a60-b586-ee715cd9f34b_aug.csv'

df_aug = pd.read_csv(file_path_a)

file_path_s = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/e95523e6-3470-4a60-b586-ee715cd9f34b_sept.csv'

df_sep = pd.read_csv(file_path_s)

file_path_o = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/e95523e6-3470-4a60-b586-ee715cd9f34b_oct.csv'

df_oct = pd.read_csv(file_path_o)

# Combine vertically
df_a_s_o = pd.concat([df_aug, df_sep, df_oct], axis=0, ignore_index=True)

print(df_a_s_o)

In [ ]:
df_a_s_o.info()

In [ ]:
# Data Check

# Calculate the overall count of unique user IDs
unique_user_count = df_a_s_o['user_id'].nunique()

# Calculate the sum of unit_a
sum_of_unit_a = df_a_s_o['unit_a'].sum()

# Calculate the sum of watt_h
sum_of_unit_wh = df_a_s_o['unit_wh'].sum()

# Print the results
print(f"Unique User Count: {unique_user_count}")
print(f"Sum of unit_a: {sum_of_unit_a}")
print(f"Sum of unit_wh: {sum_of_unit_wh}")

# Unique User Count: 1028
# Sum of unit_a: 84714332.39000002
# Sum of unit_wh: 57182938816884.78


In [ ]:
# Decorate data with engineered values

from datetime import datetime
import pytz

# Function to convert to PST and extract datetime
def convert_to_pst_as_datetime(timestamp):
    # Parse the UTC timestamp
    utc_time = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    # Set timezone to UTC
    utc_time = utc_time.replace(tzinfo=pytz.UTC)
    # Convert to PST
    pst_time = utc_time.astimezone(pytz.timezone('US/Pacific'))
    # Truncate to day, month, year, and hour (zero minutes and seconds)
    return pst_time.replace(minute=0, second=0, microsecond=0)

# Apply the function to convert timestamp
df_a_s_o['time_sample'] = df_a_s_o['timestamp'].apply(convert_to_pst_as_datetime)

# Add a column for day of the week (0 = Monday, 6 = Sunday)
df_a_s_o['day_of_week'] = df_a_s_o['time_sample'].dt.dayofweek

# Add a column for hour of the day (24hr format)
df_a_s_o['hour_of_day'] = df_a_s_o['time_sample'].dt.hour

# Add a column for ISO week number
df_a_s_o['week_number'] = df_a_s_o['time_sample'].dt.isocalendar().week

# Add in count of unique users
df_a_s_o['unique_user_count'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['user_id']
    .transform('nunique')
)

# Add in sum of unit_a
df_a_s_o['sum_of_unit_a'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['unit_a']
    .transform('sum')
)

# Add in sum of watt_h
df_a_s_o['sum_of_unit_wh'] = (
    df_a_s_o
    .groupby(['week_number', 'day_of_week', 'hour_of_day'])['unit_wh']
    .transform('sum')
)

# Print the updated DataFrame
print(df_a_s_o)

In [ ]:
# Data Check
print(df_a_s_o['week_number'].unique())


# Calculate the overall count of unique user IDs
unique_user_count = df_a_s_o['user_id'].nunique()

# Calculate the sum of unit_a
sum_of_unit_a = df_a_s_o['unit_a'].sum()

# Calculate the sum of watt_h
sum_of_unit_wh = df_a_s_o['unit_wh'].sum()

# Print the results
print(f"Unique User Count: {unique_user_count}")
print(f"Sum of unit_a: {sum_of_unit_a}")
print(f"Sum of unit_wh: {sum_of_unit_wh}")

# Unique User Count: 1028
# Sum of unit_a: 84714332.39000002
# Sum of unit_wh: 57182938816884.78

In [ ]:
df_a_s_o.info()

In [ ]:
# Reduce the df
df_a_s_o.drop(columns=['_time','transactionId','timestamp','station_id','serial_number','cluster','time_sample'], inplace=True)


In [ ]:
# Reduce the DataFrame to unique rows based on the specified columns
reduced_df = df_a_s_o.drop_duplicates(
    subset=['day_of_week', 'hour_of_day', 'week_number', 'unique_user_count', 'sum_of_unit_a', 'sum_of_unit_wh']
)

# Keep only the specified columns
reduced_df = reduced_df[['day_of_week', 'hour_of_day', 'week_number', 'unique_user_count', 'sum_of_unit_a', 'sum_of_unit_wh']]

# Display the resulting DataFrame
print(reduced_df.info())
print(reduced_df.head())


In [ ]:

# Calculate the sum of unit_a
sum_of_unit_a = reduced_df['sum_of_unit_a'].sum()

# Calculate the sum of watt_h
sum_of_unit_wh = reduced_df['sum_of_unit_wh'].sum()

# Print the results

print(f"Sum of unit_a: {sum_of_unit_a}")
print(f"Sum of unit_wh: {sum_of_unit_wh}")

# Unique User Count: 1028
# Sum of unit_a: 84714332.39000002
# Sum of unit_wh: 57182938816884.78



In [ ]:
# Write a local file to take a look

df_a_s_o.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/df_a_s_o.csv', index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.regplot(x='unique_user_count', y='sum_of_unit_wh', data=df_a_s_o, scatter_kws={'alpha': 0.3}, line_kws={'color': 'red'})
plt.xlabel('User unique_user_count Count')
plt.ylabel('Total Unit WH')
plt.title('Regression Plot: User ID Count vs. Total Unit WH')
plt.show()

In [ ]:
df_a_s_o.info()

In [ ]:
# Data is week 32 through week 44 (12)
# So below, there is no week lag1 value for week 32 because it is the first

# Identify the peak total_unit_wh for each week
peak_weekly_data = df.loc[df.groupby('week_number')['sum_of_unit_wh'].idxmax()]

# Sort by week number to ensure correct lagging
peak_weekly_data = peak_weekly_data.sort_values('week_number')

# Add only lag_1 features
peak_weekly_data['lag_1_day_of_week'] = peak_weekly_data['day_of_week'].shift(1)
peak_weekly_data['lag_1_hour'] = peak_weekly_data['hour_of_day'].shift(1)

# Drop rows with insufficient lag (week 1)
peak_weekly_data = peak_weekly_data.dropna()

# Retain only relevant columns
peak_weekly_data = peak_weekly_data[['week_number', 'day_of_week', 'hour_of_day', 'lag_1_day_of_week', 'lag_1_hour']]

print("Updated DataFrame:")
print(peak_weekly_data)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Features (lagged day of week and hour) and target (day of week)
X = peak_weekly_data[['lag_1_day_of_week', 'lag_1_hour']]
y = peak_weekly_data['day_of_week']  # Target: Day of the week

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Day of Week Prediction Accuracy:", accuracy)

# Display true vs predicted values
results = pd.DataFrame({'True Day': y_test, 'Predicted Day': y_pred})
print("\nTrue vs Predicted Days of the Week:")
print(results)


In [ ]:


# Feature importance for day_of_week classification
clf_importances = clf.feature_importances_
plt.barh(X.columns, clf_importances)
plt.title("Feature Importance for Day of Week Prediction")
plt.show()

# Feature importance for hour regression
reg_importances = reg.feature_importances_
plt.barh(X.columns, reg_importances)
plt.title("Feature Importance for Hour Prediction")
plt.show()


## Extract from Eddie

In [ ]:
file_path_a = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/2024-08-01.csv'

df_big = pd.read_csv(file_path_a)



### Data Exploration

In [ ]:
df_big.info()

In [ ]:
header = [
    "qrcode",  # Column 0
    "connector",  # Column 1
    "serial_num",  # Column 2
    "org_id",  # Column 3
    "property_id",  # Column 4
    "station_id",  # Column 5
    "transaction_id",  # Column 6
    "metered_type",  # Column 7
    "timestamp",  # Column 8
    "metered_value"   # Column 9
]

df_big.columns = header

In [ ]:
df_big['Timestamp'] = pd.to_datetime(df_big['Timestamp'])


## List of tables I can access

In [ ]:

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Execute a query to fetch all table names
    query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
    """

    cursor.execute(query)
    tables = cursor.fetchall()

    # Print the table names
    for table in tables:
        print(table[0])

except Exception as error:
    print(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        print("Connection closed.")


## Look at each table I can access

In [ ]:
# This creates a table of field names and sample values
import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# List of tables to process
tables = [
    "group_discount_properties", "adjustment", "payment", "group_discount",
    "roles", "pos_device", "location", "station_credit_program", "station_history",
    "subscription", "user_discount_properties", "users", "router", "vehicle",
    "properties", "rfid_user", "stripe_payment_intent", "adr", "audit",
    "cluster_name", "global_setting", "station_logs", "station_model",
    "awsdms_ddl_audit", "user_access", "pricing", "stations", "gateway",
    "gateway_ip_lease", "errors", "credit_program", "maintenance_window",
    "refresh_token", "net_device_ip_lease", "ocpp_sub_session", "property_types",
    "user_device", "transaction", "address", "accounts", "net_devices",
    "organizations", "ocpp_session", "panels", "flyway_schema_history",
    "connectors", "clusters"
]


# "station_logs" is the big one. WOuld have the same fields/data as MeterValues data in Splunk.




# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Query to fetch the first few rows from the current table
        query = f"SELECT * FROM {table} LIMIT 10;"
        cursor.execute(query)

        # Fetch the rows
        rows = cursor.fetchall()
        # Fetch the column headers
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame from the fetched data
        df = pd.DataFrame(rows, columns=column_names)

        # Prepare the transposed DataFrame
        transposed_data = {
            'Header': column_names,
            'Data Type': [df[col].dtype.name for col in column_names],  # Get the data type
            'Example': [df[col].iloc[0] if not df[col].empty else None for col in column_names]  # Example from the first row
        }

        df_transposed = pd.DataFrame(transposed_data)

        # Write the DataFrame to CSV
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/{table}_fields.csv'
        df_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")

In [ ]:
# This creates a table of sample records

import os
import logging
import psycopg2
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection parameters from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')  # Fetch the port from environment variables
}

# Function to fetch non-transposed data
def fetch_non_transposed_data(cursor, table):
    query = f"SELECT * FROM {table} LIMIT 10;"
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(rows, columns=column_names)
    return df

# List of tables to process
tables = [
    "group_discount_properties", "adjustment", "payment", "group_discount",
    "roles", "pos_device", "location", "station_credit_program", "station_history",
    "subscription", "user_discount_properties", "users", "router", "vehicle",
    "properties", "rfid_user", "stripe_payment_intent", "adr", "audit",
    "cluster_name", "global_setting", "station_logs", "station_model",
    "awsdms_ddl_audit", "user_access", "pricing", "stations", "gateway",
    "gateway_ip_lease", "errors", "credit_program", "maintenance_window",
    "refresh_token", "net_device_ip_lease", "ocpp_sub_session", "property_types",
    "user_device", "transaction", "address", "accounts", "net_devices",
    "organizations", "ocpp_session", "panels", "flyway_schema_history",
    "connectors", "clusters"
]

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**connection_params)
    cursor = connection.cursor()

    # Loop through each table name
    for table in tables:
        logging.info(f"Processing table: {table}")

        # Fetch non-transposed data
        df_non_transposed = fetch_non_transposed_data(cursor, table)

        # Write the DataFrame to CSV with new naming convention
        output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/{table}_example_data.csv'
        df_non_transposed.to_csv(output_csv_path, index=False)
        logging.info(f"Data written to {output_csv_path} successfully.")

except Exception as error:
    logging.error(f"Error connecting to the database: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        logging.info("Connection closed.")


# Now I need to build the correct table directly from RS

In [ ]:
import os
import pandas as pd
import logging
from itertools import combinations

# Configure logging
logging.basicConfig(level=logging.INFO)

# Path to the directory containing the CSV files
data_dir = '/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/'

# List of tables (as per your previous code)
tables = [
    "group_discount_properties", "adjustment", "payment", "group_discount",
    "roles", "pos_device", "location", "station_credit_program", "station_history",
    "subscription", "user_discount_properties", "users", "router", "vehicle",
    "properties", "rfid_user", "stripe_payment_intent", "adr", "audit",
    "cluster_name", "global_setting", "station_logs", "station_model",
    "awsdms_ddl_audit", "user_access", "pricing", "stations", "gateway",
    "gateway_ip_lease", "errors", "credit_program", "maintenance_window",
    "refresh_token", "net_device_ip_lease", "ocpp_sub_session", "property_types",
    "user_device", "transaction", "address", "accounts", "net_devices",
    "organizations", "ocpp_session", "panels", "flyway_schema_history",
    "connectors", "clusters"
]

# Function to load CSV files into DataFrames
def load_dataframes(tables):
    dataframes = {}
    for table in tables:
        csv_path = os.path.join(data_dir, f"{table}_example_data.csv")
        try:
            df = pd.read_csv(csv_path)
            dataframes[table] = df
            logging.info(f"Loaded data for table: {table}")
        except Exception as e:
            logging.error(f"Error loading data for table {table}: {e}")
    return dataframes

# Function to find strict join matches
def find_strict_joins(df1, df2, table1_name, table2_name):
    strict_joins = []
    # Iterate over all column pairs
    for col1 in df1.columns:
        for col2 in df2.columns:
            if df1[col1].dtype == df2[col2].dtype:
                # Perform the join
                joined_df = pd.merge(df1, df2, left_on=col1, right_on=col2, how='inner')
                # Check if all rows in df1 are in the joined DataFrame
                if len(joined_df) == len(df1):
                    strict_joins.append((col1, col2))
                    logging.info(f"Strict join success: {table1_name}.{col1} <-> {table2_name}.{col2}")
    return strict_joins

# Main function to perform the strict join analysis
def analyze_strict_joins(tables):
    dataframes = load_dataframes(tables)
    results = {}
    table_pairs = combinations(tables, 2)

    for table1, table2 in table_pairs:
        df1 = dataframes.get(table1)
        df2 = dataframes.get(table2)

        if df1 is not None and df2 is not None:
            logging.info(f"Analyzing strict joins between {table1} and {table2}")
            joins = find_strict_joins(df1, df2, table1, table2)
            if joins:
                results[f"{table1} <-> {table2}"] = joins
        else:
            logging.warning(f"Data for {table1} or {table2} is missing. Skipping.")

    return results

# Run the strict join analysis
strict_join_results = analyze_strict_joins(tables)

# Print the results
for table_pair, joins in strict_join_results.items():
    print(f"\nStrict joins for {table_pair}:")
    for col1, col2 in joins:
        print(f"Columns: {table_pair.split(' <-> ')[0]}.{col1} <-> {table_pair.split(' <-> ')[1]}.{col2}")

if not strict_join_results:
    print("No strict joins found.")


In [ ]:
import os
import logging
import pandas as pd
from sqlalchemy import create_engine

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load credentials from file
def load_credentials(path_to_credentials):
    try:
        with open(path_to_credentials, 'r') as file:
            for line_num, line in enumerate(file, start=1):
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)  # Split only on the first '='
                    os.environ[key.strip()] = value.strip()
                else:
                    logging.warning(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")
        logging.info("Credentials loaded successfully.")
    except Exception as e:
        logging.error(f"Error loading credentials: {str(e)}")

# Call the function to load credentials
path_to_credentials = '/content/drive/MyDrive/Colab Notebooks/credentials/aws_credentials.txt'
load_credentials(path_to_credentials)

# Create connection string for SQLAlchemy
connection_string = f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(connection_string)

# Function to fetch column names for a table
def get_columns(table_name):
    try:
        query = f"""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{table_name}';
        """
        with engine.connect() as connection:
            df = pd.read_sql_query(query, connection)
        return df[['column_name', 'data_type']].to_dict('records')
    except Exception as e:
        logging.error(f"Error fetching columns for table {table_name}: {e}")
        return []

# Function to test join logic between two tables
def test_joins(table1, table2, attempts=3):
    columns_table1 = get_columns(table1)
    columns_table2 = get_columns(table2)
    successful_joins = []

    for col1 in columns_table1:
        for col2 in columns_table2:
            # Only test joins on matching data types
            if col1['data_type'] == col2['data_type']:
                success_count = 0
                for _ in range(attempts):  # Attempt the join multiple times
                    query = f"""
                    SELECT *
                    FROM {table1} t1
                    INNER JOIN {table2} t2
                    ON t1.{col1['column_name']} = t2.{col2['column_name']}
                    LIMIT 1;  -- Test with one row at a time
                    """
                    try:
                        with engine.connect() as connection:
                            df = pd.read_sql_query(query, connection)
                            if not df.empty:
                                success_count += 1
                    except Exception as e:
                        logging.debug(f"Join failed for {table1}.{col1['column_name']} = {table2}.{col2['column_name']}: {e}")

                if success_count == attempts:  # Only count as successful if all attempts work
                    successful_joins.append((col1['column_name'], col2['column_name']))
                    logging.info(f"Successful join: {table1}.{col1['column_name']} = {table2}.{col2['column_name']}")

    return successful_joins

# Cross-check join fields for all table pairs
tables = [
    "users", "ocpp_sub_session"
]

results = {}

for i, table1 in enumerate(tables):
    for table2 in tables[i+1:]:
        logging.info(f"Testing joins between {table1} and {table2}")
        joins = test_joins(table1, table2)
        if joins:
            results[f"{table1} <-> {table2}"] = joins
        else:
            logging.info(f"No join found between {table1} and {table2}")

# Print results
for table_pair, joins in results.items():
    print(f"Successful joins for {table_pair}: {joins}")

if not results:
    print("No successful joins found.")


In [ ]:
import pandas as pd
import json

# Example dataframe (assuming df['message'] contains the raw strings)
# Clean the 'message' column by removing the prefix 'OCPP : MeterValues '
def clean_message(msg):
    try:
        # Remove the prefix
        msg_cleaned = msg.lstrip('OCPP : MeterValues ')

        # Attempt to load the cleaned message as JSON
        return json.loads(msg_cleaned)
    except (json.JSONDecodeError, TypeError):
        # If the message cannot be decoded as JSON, return None or handle as needed
        return None

# Apply the function to the 'message' column
df['message'] = df['message'].apply(clean_message)

# Filter out rows where the 'message' column is None (indicating a JSON parse failure)
df = df[df['message'].notna()]

# Step 1: Extract top-level fields and keep 'meterValue' as is (as a list of dicts)
flattened_rows = []

for idx, row in df.iterrows():
    message = row['message']  # Now this is a valid JSON object

    # Extract top-level fields
    connector_id = message.get('connectorId')
    transaction_id = message.get('transactionId')

    # Keep the 'meterValue' field as is (as a list of dicts)
    meter_value = message.get('meterValue', [])

    # Add a row to the flattened list, including the nested 'meterValue' list
    flattened_rows.append({
        '_time': row['time'],  # Retain the original timestamp from the dataframe
        'user_id': row['user_id'],  # Assuming 'user_id' is part of the original dataframe
        'connectorId': connector_id,
        'meterValue': meter_value  # The entire 'meterValue' field, as it is (list of dictionaries)
    })

# Step 2: Create a new DataFrame from the flattened rows
flattened_df = pd.DataFrame(flattened_rows)

# Display the resulting DataFrame
print(flattened_df.head())


In [ ]:
import pandas as pd

# Set pandas options to display the full content of any column (e.g., 'meterValue')
pd.set_option('display.max_colwidth', None)

# Now, display the full content of the 'meterValue' column for the first 5 rows
print(flattened_df['meterValue'].head(1))

In [ ]:
import pandas as pd

# Create a list to hold the expanded rows
expanded_rows = []

# Iterate over each row in the dataframe
for idx, row in flattened_df.iterrows():
    meter_values = row['meterValue']  # This is the list of meter readings (list of dicts)

    # For each meter value entry (there should be one timestamp and a list of measurements)
    for meter in meter_values:
        timestamp = meter['timestamp']  # Extract the timestamp

        # Initialize values for each measurement type
        watt_hours_value = None  # WattHours
        amps_value = None        # Amps (Current)
        voltage_value = None     # Voltage (Volts)

        # Iterate over the sampledValue list (which contains the three measurements)
        for sample in meter['sampledValue']:
            # Check the 'unit' to assign the value to the correct column
            if sample['unit'] == 'Wh':  # WattHours
                watt_hours_value = sample['value']
            elif sample['unit'] == 'A':  # Amps (Current)
                amps_value = sample['value']
            elif sample['unit'] == 'V':  # Volts (Voltage)
                voltage_value = sample['value']

        # Append the expanded row with the extracted values
        expanded_rows.append({
            '_time': row['_time'],  # Retain the original timestamp from the dataframe
            'user_id': row['user_id'],  # Assuming 'user_id' is part of the original dataframe
            'connectorId': row['connectorId'],  # Connector ID
            'timestamp': timestamp,  # Timestamp from the meter value
            'WattHours': watt_hours_value,  # Renamed to WattHours
            'Amps': amps_value,  # Keep Amps as the column name
            'Voltage': voltage_value  # Value for Voltage (V)
        })

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

# Convert the numeric columns to appropriate types (float)
expanded_df['WattHours'] = pd.to_numeric(expanded_df['WattHours'], errors='coerce')
expanded_df['Amps'] = pd.to_numeric(expanded_df['Amps'], errors='coerce')
expanded_df['Voltage'] = pd.to_numeric(expanded_df['Voltage'], errors='coerce')

# Display the resulting DataFrame
print(expanded_df.head())


In [ ]:
# Ensure all columns are numeric (in case there are any string values left)
expanded_df['WattHours'] = pd.to_numeric(expanded_df['WattHours'], errors='coerce')
expanded_df['Amps'] = pd.to_numeric(expanded_df['Amps'], errors='coerce')
expanded_df['Voltage'] = pd.to_numeric(expanded_df['Voltage'], errors='coerce')

# Classify values as 0 or > 0 for each of the measurements
expanded_df['WattHours_Class'] = expanded_df['WattHours'].apply(lambda x: '0' if x == 0 else '>0')
expanded_df['Amps_Class'] = expanded_df['Amps'].apply(lambda x: '0' if x == 0 else '>0')
expanded_df['Voltage_Class'] = expanded_df['Voltage'].apply(lambda x: '0' if x == 0 else '>0')

# Set up the plot
plt.figure(figsize=(18, 6))

# Plot the count of each class for 'WattHours', 'Amps', and 'Voltage'
plt.subplot(1, 3, 1)
sns.countplot(data=expanded_df, x='WattHours_Class')
plt.title('Count of Rows with WattHours: 0 vs > 0')
plt.xlabel('WattHours Class')
plt.ylabel('Count')

plt.subplot(1, 3, 2)
sns.countplot(data=expanded_df, x='Amps_Class')
plt.title('Count of Rows with Amps: 0 vs > 0')
plt.xlabel('Amps Class')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
sns.countplot(data=expanded_df, x='Voltage_Class')
plt.title('Count of Rows with Voltage: 0 vs > 0')
plt.xlabel('Voltage Class')
plt.ylabel('Count')

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure that the '_time' column is in datetime format
expanded_df['timestamp'] = pd.to_datetime(expanded_df['timestamp'], errors='coerce')

# Convert 'Amps', 'WattHours', and 'Voltage' to numeric (handling any errors)
expanded_df['Amps'] = pd.to_numeric(expanded_df['Amps'], errors='coerce')
expanded_df['WattHours'] = pd.to_numeric(expanded_df['WattHours'], errors='coerce')
expanded_df['Voltage'] = pd.to_numeric(expanded_df['Voltage'], errors='coerce')

# Drop rows where any of the values are missing
expanded_df = expanded_df.dropna(subset=['_time', 'Amps', 'WattHours', 'Voltage'])

# Set the style for the plots
sns.set(style="whitegrid")

# Create the figure and axes for the plots
plt.figure(figsize=(18, 6))

# Plot Amps over time
plt.subplot(1, 3, 1)
plt.plot(expanded_df['timestamp'], expanded_df['Amps'], label='Amps', color='b', alpha=0.7)
plt.title('Amps over Time')
plt.xlabel('Time')
plt.ylabel('Amps')
plt.xticks(rotation=45)

# Plot WattHours over time
plt.subplot(1, 3, 2)
plt.plot(expanded_df['timestamp'], expanded_df['WattHours'], label='WattHours', color='g', alpha=0.7)
plt.title('WattHours over Time')
plt.xlabel('Time')
plt.ylabel('WattHours')
plt.xticks(rotation=45)

# Plot Voltage over time
plt.subplot(1, 3, 3)
plt.plot(expanded_df['timestamp'], expanded_df['Voltage'], label='Voltage', color='r', alpha=0.7)
plt.title('Voltage over Time')
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.xticks(rotation=45)

# Adjust layout to avoid overlap of labels
plt.tight_layout()
plt.show()


In [ ]:
expanded_df.info()

In [ ]:
# Run descriptive statistics on 'Amps', 'WattHours', and 'Voltage'
descriptive_stats = expanded_df[['Amps', 'WattHours', 'Voltage']].describe()

# Display the statistics
print(descriptive_stats)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for the plots
sns.set(style="whitegrid")

# Create a figure with 3 subplots (1 row, 3 columns)
plt.figure(figsize=(18, 6))

# Plot for Amps
plt.subplot(1, 3, 1)
sns.boxplot(data=expanded_df['Amps'], color='skyblue')
plt.title('Boxplot of Amps')
plt.ylabel('Amps')

# Plot for WattHours
plt.subplot(1, 3, 2)
sns.boxplot(data=expanded_df['WattHours'], color='lightgreen')
plt.title('Boxplot of WattHours')
plt.ylabel('WattHours')

# Plot for Voltage
plt.subplot(1, 3, 3)
sns.boxplot(data=expanded_df['Voltage'], color='lightcoral')
plt.title('Boxplot of Voltage')
plt.ylabel('Voltage')

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()


In [ ]:
# count of propertyIDs

import os
import psycopg2

# Load credentials from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Table to process (the 'stations' table)
table = "stations"

# Connect to the PostgreSQL database and run the query
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Query to count unique 'property_id' values
query = f"SELECT COUNT(DISTINCT property_id) FROM {table};"
cursor.execute(query)

# Fetch the result
result = cursor.fetchone()

# Extract and print the count
unique_property_id_count = result[0] if result else 0
print(f"Number of unique 'property_id' values: {unique_property_id_count}")

# Clean up
cursor.close()
connection.close()

In [ ]:
#Count of cluster IDs

import os
import psycopg2

# Load credentials from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Table to process (the 'stations' table)
table = "stations"

# Connect to the PostgreSQL database and run the query
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Query to count unique 'property_id' values
query = f"SELECT COUNT(DISTINCT cluster_id) FROM {table};"
cursor.execute(query)

# Fetch the result
result = cursor.fetchone()

# Extract and print the count
unique_property_id_count = result[0] if result else 0
print(f"Number of unique 'cluster_id' values: {unique_property_id_count}")

# Clean up
cursor.close()
connection.close()

In [ ]:
# counts of peropertyID and clusterIDimport os
import psycopg2

# Load credentials from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Table to process (the 'stations' table)
table = "stations"

# Connect to the PostgreSQL database and run the query
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Query to count unique 'cluster_id' values for each 'property_id'
query = f"""
SELECT property_id, COUNT(DISTINCT cluster_id)
FROM {table}
GROUP BY property_id
;
"""
cursor.execute(query)

# Fetch all the results
results = cursor.fetchall()

# Print the results
for row in results:
    property_id, cluster_count = row
    print(f"Property ID: {property_id}, Unique Cluster ID Count: {cluster_count}")

# Clean up
cursor.close()
connection.close()

import os
import psycopg2

# Load credentials from environment variables
connection_params = {
    'host': os.getenv('DB_HOST'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'port': os.getenv('DB_PORT')
}

# Table to process (the 'stations' table)
table = "stations"

# Connect to the PostgreSQL database and run the query
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

# Query to count unique 'property_id' values for each 'cluster_id'
query = f"""
SELECT cluster_id, COUNT(DISTINCT property_id)
FROM {table}
GROUP BY cluster_id;
"""
cursor.execute(query)

# Fetch all the results
results = cursor.fetchall()

# Print the results
for row in results:
    cluster_id, property_count = row
    print(f"Cluster ID: {cluster_id}, Unique Property ID Count: {property_count}")

# Clean up
cursor.close()
connection.close()


OCPP_SessionID has a userID and TransactionID
I need to map to the cluster and property

Stations has propertyID and cluster_id




In [ ]:
# Clean message field and port to a df
import json
import pandas as pd

# Function to clean up the 'message' field by removing the prefix and parsing JSON
def clean_and_parse_message(message):
    try:
        # Strip the non-JSON prefix before the first '{'
        cleaned_message = message[message.find('{'):]
        # Parse the cleaned JSON string
        return json.loads(cleaned_message)
    except json.JSONDecodeError as e:
        print(f"Invalid JSON in message: {message}\nError: {e}")
        return None

# Function to flatten nested JSON
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if isinstance(x, dict):
            for a in x:
                flatten(x[a], name + a + '_')
        elif isinstance(x, list):
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Apply the cleaning and parsing function to all rows in the 'message' field
df['parsed_message'] = df['message'].apply(clean_and_parse_message)

# Drop rows where parsing failed (invalid JSON) or was not cleaned properly
valid_df = df[df['parsed_message'].notnull()]

# Flatten all the JSON objects and store them in a new DataFrame
flattened_data = valid_df['parsed_message'].apply(flatten_json).apply(pd.Series)

# Combine the flattened JSON fields with the original DataFrame (excluding the original 'message' and 'parsed_message' fields)
new_df = pd.concat([valid_df.drop(columns=['message', 'parsed_message']), flattened_data], axis=1)

# Write the DataFrame to CSV with new naming convention
output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/new_df.csv'
new_df.to_csv(output_csv_path, index=False)
logging.info(f"Data written to {output_csv_path} successfully.")




In [ ]:
new_df.info
new_df.head()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df= new_df

# Assuming your DataFrame is named df
# Step 1: Convert 'time' to datetime
df['time'] = pd.to_datetime(df['time'], errors='coerce')  # errors='coerce' will turn invalid parsing to NaT

# Step 2: Convert 'meterValue_0_sampledValue_0_value' to numeric
df['meterValue_0_timestamp'] = pd.to_numeric(df['meterValue_0_timestamp'], errors='coerce')

# Step 3: Drop any rows with NaT or NaN values (optional, depending on your needs)
df = df.dropna(subset=['time', 'meterValue_0_sampledValue_0_value'])

# Step 4: Plot the time series
plt.figure(figsize=(10, 6))
plt.plot(df['meterValue_0_timestamp'], df['meterValue_0_sampledValue_0_value'], label='Meter Value', color='b')
plt.xlabel('Time')
plt.ylabel('Meter Value')
plt.title('Meter Value Over Time')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# Group by user_id and count the occurrences of meterValue_0_timestamp
hist_data = df.groupby('user_id')['message'].count().reset_index()

# Rename the columns for clarity
hist_data.columns = ['user_id', 'count']

# Sort the data by user_id for better visualization
hist_data = hist_data.sort_values('user_id')

# Plotting the normal line plot
plt.figure(figsize=(10, 6))
plt.plot(hist_data['user_id'], hist_data['count'], marker='o', linestyle='-', color='skyblue')
plt.xlabel('User ID')
plt.ylabel('Count of message')
plt.title('Count of message per User ID')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
import pandas as pd

# Assuming new_df is already defined and contains the necessary columns

# List of columns to keep
columns_to_keep = [
    #'time',
    'user_id',
    #'station_id',
    'property_id',
    'connectorId',
    'meterValue_0_timestamp',
    'meterValue_0_sampledValue_1_value',
    'meterValue_0_sampledValue_1_context',
    'meterValue_0_sampledValue_1_format',
    'meterValue_0_sampledValue_1_measurand',
    'meterValue_0_sampledValue_1_phase',
    'meterValue_0_sampledValue_1_location',
    'meterValue_0_sampledValue_1_unit'
]

# Create new_df_2 with only the selected columns
new_df_2 = new_df[columns_to_keep].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Convert 'time' to datetime
new_df_2['meterValue_0_timestamp'] = pd.to_datetime(new_df_2['meterValue_0_timestamp'], errors='coerce')

# Check for any NaT values that may have resulted from the conversion
if new_df_2['meterValue_0_timestamp'].isnull().any():
    print("Some values could not be converted to datetime.")

# Extract day and hour using .loc to avoid warnings
new_df_2.loc[:, 'meterValue_0_day'] = new_df_2['meterValue_0_timestamp'].dt.date
new_df_2.loc[:, 'meterValue_0_hour'] = new_df_2['meterValue_0_timestamp'].dt.hour



In [ ]:
new_df_2.info()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = new_df_2

# Assuming df is your DataFrame
# Group by user_id and count the occurrences of meterValue_0_timestamp
hist_data = df.groupby('user_id')['meterValue_0_timestamp'].count().reset_index()

# Rename the columns for clarity
hist_data.columns = ['user_id', 'count']

# Sort the data by user_id for better visualization
hist_data = hist_data.sort_values('user_id')

# Plotting the normal line plot
plt.figure(figsize=(10, 6))
plt.plot(hist_data['user_id'], hist_data['count'], marker='o', linestyle='-', color='skyblue')
plt.xlabel('User ID')
plt.ylabel('Count of message')
plt.title('Count of message per User ID')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
# Assuming new_df_2 is the df

unique_values = new_df_2['user_id'].unique()

# To display the unique values
print(unique_values)


# Assuming new_df_2 is your DataFrame
unique_count = new_df_2['user_id'].nunique()

# To display the count of unique user_id values
print(f"Number of unique user_id values: {unique_count}")

In [ ]:
import pandas as pd


new_df_2['meterValue_0_sampledValue_1_value'] = pd.to_numeric(new_df_2['meterValue_0_sampledValue_1_value'], errors='coerce')

max_values = new_df_2.loc[new_df_2.groupby(['user_id', 'meterValue_0_day'])['meterValue_0_sampledValue_1_value'].idxmax()]

result_df = max_values[['user_id', 'meterValue_0_day', 'meterValue_0_sampledValue_1_value', 'meterValue_0_timestamp']]

print(result_df)
result_df.info()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# Group by user_id and count the occurrences of meterValue_0_timestamp
hist_data = df.groupby('user_id')['meterValue_0_timestamp'].count().reset_index()

# Rename the columns for clarity
hist_data.columns = ['user_id', 'count']

# Sort the data by user_id for better visualization
hist_data = hist_data.sort_values('user_id')

# Plotting the normal line plot
plt.figure(figsize=(10, 6))
plt.plot(hist_data['user_id'], hist_data['count'], marker='o', linestyle='-', color='skyblue')
plt.xlabel('User ID')
plt.ylabel('Count of meterValue_0_timestamp')
plt.title('Count of meterValue_0_timestamp per User ID')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming result_df is your DataFrame

# Print column names to verify
print("Column names in DataFrame:")
print(result_df.columns.tolist())

# Specify the user_id you're interested in
specific_user_id = '013f0335-da69-4fdd-b378-b6a9a8cfc8a8'  # replace with the actual user_id

# Filter the DataFrame for the specific user_id
filtered_df = result_df[result_df['user_id'] == specific_user_id]

# Check if there are any rows for the specified user_id
if not filtered_df.empty:
    # Check for the timestamp column again
    timestamp_col = 'meterValue_0_timestamp'  # Update if necessary
    value_col = 'meterValue_0_sampledValue_1_value'

    # Ensure the column names are correct
    print("Filtered DataFrame columns:")
    print(filtered_df.columns.tolist())

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(filtered_df[timestamp_col], filtered_df[value_col], marker='o')
    plt.title(f'Meter Values for User ID: {specific_user_id}')
    plt.xlabel('Timestamp')
    plt.ylabel('Meter Value')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
    plt.grid()
    plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
    plt.show()
else:
    print(f"No data found for user_id: {specific_user_id}")


In [ ]:

# Write the DataFrame to CSV
output_csv_path = f'/content/drive/MyDrive/Colab Notebooks/Data_sets/Chargie/result_df_exported.csv'
result_df.to_csv(output_csv_path, index=False)
logging.info(f"Data written to {output_csv_path} successfully.")




In [ ]:
# Identify source of error in message JSON
import json

# Function to recursively extract all keys
def get_all_keys(data, keys=set()):
    if isinstance(data, dict):
        for key, value in data.items():
            keys.add(key)
            get_all_keys(value, keys)
    elif isinstance(data, list):
        for item in data:
            get_all_keys(item, keys)
    return keys

# Initialize set to collect unique keys
all_keys = set()

# Variable to track the first invalid JSON example
invalid_json_index = None
invalid_json_sample = None

# Iterate through df['message'] and try to process each row
for index, message in enumerate(df['message']):
    try:
        # Parse JSON string
        message_json = json.loads(message)
        # Extract keys from each valid message
        all_keys = get_all_keys(message_json, all_keys)
    except json.JSONDecodeError:
        # Capture the index and value of the first invalid JSON row
        invalid_json_index = index
        invalid_json_sample = df.iloc[index]
        break  # Stop after finding the first invalid JSON entry

# Print all extracted keys
print("Extracted keys:", all_keys)

# Display the first invalid JSON example
if invalid_json_sample is not None:
    print(f"First invalid JSON entry at row {invalid_json_index}:")
    print(invalid_json_sample)
else:
    print("No invalid JSON entries found.")


In [ ]:
# Print the first 5 rows of the 'message' field to inspect for any issues
for i in range(5):
    print(f"Row {i} message: {df['message'].iloc[i]}\n")


In [ ]:
df.info()
meterValue_0_sampledValue_1_unit = A

In [ ]:
import json

# Function to clean up the 'message' field by removing the prefix and parsing JSON
def clean_and_parse_message(message):
    try:
        # Strip the non-JSON prefix before the first '{'
        cleaned_message = message[message.find('{'):]  # This removes everything before the first '{'
        # Parse the cleaned JSON string
        return json.loads(cleaned_message)
    except json.JSONDecodeError as e:
        print(f"Invalid JSON in message: {message}\nError: {e}")
        return None

# Function to recursively extract all keys from JSON objects
def get_all_keys(data, keys=set()):
    if isinstance(data, dict):
        for key, value in data.items():
            keys.add(key)
            get_all_keys(value, keys)
    elif isinstance(data, list):
        for item in data:
            get_all_keys(item, keys)
    return keys

# Apply the cleaning and parsing function to all rows in the 'message' field
df['parsed_message'] = df['message'].apply(clean_and_parse_message)

# Initialize a set to store unique keys from the parsed JSON objects
json_keys = set()

# Extract keys from the 'parsed_message' field for all valid rows
for parsed_message in df['parsed_message'].dropna():  # Exclude rows with None (failed parsing)
    json_keys = get_all_keys(parsed_message, json_keys)

# Get the original DataFrame headers (columns)
df_headers = set(df.columns)

# Combine both sets: original DataFrame headers and extracted JSON keys
complete_headers = df_headers.union(json_keys)

# Print the complete view of all data fields
print("Complete view of all data fields (headers + JSON keys):")
print(complete_headers)


In [ ]:
import json

# Function to clean up the 'message' field by removing the prefix
def clean_message(message):
    # Strip the non-JSON prefix before the first '{'
    cleaned_message = message[message.find('{'):] if '{' in message else message
    return cleaned_message

# Function to validate JSON and return whether it's valid
def validate_json(message):
    try:
        # Attempt to load the cleaned message as JSON
        json.loads(message)
        return True, None  # Valid JSON
    except json.JSONDecodeError as e:
        return False, str(e)  # Invalid JSON with error message

# Apply the cleaning function to all rows in the 'message' field
df['cleaned_message'] = df['message'].apply(clean_message)

# Apply the validation function to check JSON validity and capture errors
df['is_valid_json'], df['json_validation'] = zip(*df['cleaned_message'].apply(validate_json))

# Display rows with invalid JSON
invalid_rows = df[df['is_valid_json'] == False]

# Show a few invalid rows along with the error message for investigation
print(invalid_rows[['message', 'cleaned_message', 'json_validation']].head())


In [ ]:
import json
import pandas as pd

# Function to clean up the 'message' field by removing the prefix
def clean_message(message):
    # Strip the non-JSON prefix before the first '{'
    cleaned_message = message[message.find('{'):] if '{' in message else message
    return cleaned_message

# Function to flatten nested JSON
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if isinstance(x, dict):
            for a in x:
                flatten(x[a], name + a + '_')
        elif isinstance(x, list):
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Function to clean, parse and flatten JSON data
def clean_and_flatten_message(message):
    # Clean the message by removing non-JSON prefix
    cleaned_message = clean_message(message)
    # Parse the cleaned message into JSON
    try:
        parsed_message = json.loads(cleaned_message)
        # Flatten the parsed JSON
        return flatten_json(parsed_message)
    except json.JSONDecodeError:
        return None

# Apply the clean, parse, and flatten function to all rows in the 'message' field
flattened_data = df['message'].apply(clean_and_flatten_message).apply(pd.Series)

# Combine the flattened JSON fields with the original DataFrame (excluding the original 'message' field)
flattened_df = pd.concat([df.drop(columns=['message']), flattened_data], axis=1)

# Display the first few rows of the flattened DataFrame
#print(flattened_df.head())


flattened_df.info()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#meterValue_0_sampledValue_0_unit = Wh
#meterValue_0_sampledValue_1_unit = A
#meterValue_0_sampledValue_2_unit = V
#meterValue_0_sampledValue_3_unit = Celsius


# Ensure 'time' column is in datetime format
flattened_df['time'] = pd.to_datetime(flattened_df['time'])

# Define the specific day to filter (replace with the desired date)
specific_day = '2024-10-20'  # Example date in 'YYYY-MM-DD' format

# Filter the DataFrame for the specific day
filtered_df = flattened_df[flattened_df['time'].dt.date == pd.to_datetime(specific_day).date()]

# Explicitly cast 'meterValue_0_sampledValue_2_value' to float
filtered_df['meterValue_0_sampledValue_1_value'] = filtered_df['meterValue_0_sampledValue_1_value'].astype(float)

# Create the distribution plot for the filtered data (meterValue_0_sampledValue_2_value)
plt.figure(figsize=(10, 6))
sns.kdeplot(filtered_df['meterValue_0_sampledValue_1_value'], fill=True)

# Formatting the plot
plt.xlabel('A') #Looking at amperage
plt.ylabel('Percentage (%)')
plt.title(f'Distribution of meterValue_0_sampledValue_2_value on {specific_day}')

# Convert y-axis to percentage
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0%}'.format(x)))

plt.grid(True)

# Show the plot
plt.show()


In [ ]:
df = flattened_df
df['calendar_day'] = df['time'].dt.date

In [ ]:
# Find the index of the maximum value for meterValue_0_sampledValue_1_value within each calendar_day
max_indices = df.groupby('calendar_day')['meterValue_0_sampledValue_1_value'].idxmax()

# Select the hour from the time corresponding to those indices
df['max_hour'] = df['time'].dt.hour  # Extract the hour part
max_hour_per_day = df.loc[max_indices, ['calendar_day', 'max_hour']].reset_index(drop=True)

print(max_hour_per_day)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure 'time' column is in datetime format
flattened_df['time'] = pd.to_datetime(flattened_df['time'])

# Define the specific day to filter (replace with the desired date)
specific_day = '2024-10-20'  # Example date in 'YYYY-MM-DD' format

# Filter the DataFrame for the specific day
filtered_df = flattened_df[flattened_df['time'].dt.date == pd.to_datetime(specific_day).date()]

# Explicitly cast 'meterValue_0_sampledValue_2_value' to float
filtered_df['meterValue_0_sampledValue_1_value'] = filtered_df['meterValue_0_sampledValue_1_value'].astype(float)

# Create the distribution plot for the filtered data (meterValue_0_sampledValue_2_value)
plt.figure(figsize=(10, 6))
sns.kdeplot(filtered_df['meterValue_0_sampledValue_1_value'], fill=True)

# Formatting the plot
plt.xlabel('Amps')
plt.ylabel('Percentage (%)')
plt.title(f'Distribution of meterValue_0_sampledValue_2_value on {specific_day}')

# Convert y-axis to percentage
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0%}'.format(x)))

plt.grid(True)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure 'time' column is in datetime format
flattened_df['time'] = pd.to_datetime(flattened_df['time'])

# Define the specific day to filter (replace with the desired date)
specific_day = '2024-10-20'  # Example date in 'YYYY-MM-DD' format

# Filter the DataFrame for the specific day
filtered_df = flattened_df[flattened_df['time'].dt.date == pd.to_datetime(specific_day).date()]

# Explicitly cast 'meterValue_0_sampledValue_2_value' to float
filtered_df['meterValue_0_sampledValue_2_value'] = filtered_df['meterValue_0_sampledValue_2_value'].astype(float)

# Create the distribution plot for the filtered data (meterValue_0_sampledValue_2_value)
plt.figure(figsize=(10, 6))
sns.kdeplot(filtered_df['meterValue_0_sampledValue_2_value'], fill=True)

# Formatting the plot
plt.xlabel('Volts')
plt.ylabel('Percentage (%)')
plt.title(f'Distribution of meterValue_0_sampledValue_2_value on {specific_day}')

# Convert y-axis to percentage
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0%}'.format(x)))

plt.grid(True)

# Show the plot
plt.show()
